## Unsupervised tests on codebook

In [16]:
from src.preprocessing.preprocess_ucr import UCRDatasetImporter
from src.preprocessing.data_pipeline import build_data_pipeline
from src.utils import load_yaml_param_settings
from src.models.vqvae_classification import BaseVQVAE, remap_clusters
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

from sklearn import metrics
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

In [9]:
config_dir = 'src/configs/config.yaml' #dir to config file

config = load_yaml_param_settings(config_dir)

# data pipeline
dataset_importer = UCRDatasetImporter(**config['dataset'])
batch_size = config['dataset']['batch_sizes']['vqvae']
train_data_loader, test_data_loader = [build_data_pipeline(batch_size, dataset_importer, config, kind) for kind in ['train', 'test']]

input_length = train_data_loader.dataset.X.shape[-1]

vqvae = BaseVQVAE(input_length, config)

self.X_train.shape: (375, 720)
self.X_test.shape: (375, 720)
# unique labels (train): [0 1 2]
# unique labels (test): [0 1 2]


Grabbind codebook data:

In [27]:
zqs_train = vqvae.run_through_codebook(train_data_loader)
y_train = train_data_loader.dataset.Y.flatten().astype(int)

zqs_test = vqvae.run_through_codebook(test_data_loader)
y_test = test_data_loader.dataset.Y.flatten().astype(int)

#scaling:
scaler = StandardScaler()
zqs_train = scaler.fit_transform(zqs_train)
zqs_test = scaler.fit_transform(zqs_test)

Z_train, Z_test, Y_train, Y_test = train_test_split(zqs_test, y_test, test_size=0.2)

## KNN

## SVM

In [11]:
svm = SVC(kernel='linear')
svm.fit(zqs_train, y_train)

SVC(kernel='linear')

In [12]:
y_pred = svm.predict(zqs_test)
print("Accuracy:", metrics.accuracy_score(y_true=y_test, y_pred=y_pred))

Accuracy: 0.4053333333333333


**splitting zqs_test**

In [13]:
Z_train, Z_test, Y_train, Y_test = train_test_split(zqs_test, y_test, test_size=0.2)

In [14]:
svm = SVC(kernel='linear')
svm.fit(Z_train, Y_train)
Y_pred = svm.predict(Z_test)

print("Accuracy:", metrics.accuracy_score(y_true=Y_test, y_pred=Y_pred))

Accuracy: 0.56


## Linear discriminant analysis

In [26]:
clf = LinearDiscriminantAnalysis()
clf.fit(zqs_train, y_train)

y_pred_lda = clf.predict(zqs_test)

print("accuracy:", metrics.accuracy_score(y_pred=y_pred_lda, y_true=y_test))

accuracy: 0.368


In [25]:
clf2 = LinearDiscriminantAnalysis()
clf2.fit(Z_train, Y_train)

Y_pred_lda = clf2.predict(Z_test)

print("accuracy:", metrics.accuracy_score(y_pred=Y_pred_lda, y_true=Y_test))

accuracy: 0.49333333333333335
